## Bert Classifier

In [ ]:
!pip install -q transformers

! pip install hazm

### Import Required Libraries

In [ ]:
from transformers import BertConfig, BertTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

import torch
import torch.nn as nn
import torch.nn.functional as F

import os

from hazm import word_tokenize, Normalizer
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import re
import numpy as np
import collections

### Read Data

In [ ]:
PATH = 'data/'
PATH = PATH.rstrip('/')

# Train
df_train = pd.read_csv(PATH + '/train.csv')
df_train.columns = ['index', 'comment', 'rate']

# Evaluation
df_eval = pd.read_csv(PATH + '/eval.csv')
df_eval.columns = ['index', 'comment', 'rate']

# Test
df_test = pd.read_csv(PATH + '/test.csv')
df_test.columns = ['index', 'comment', 'rate']

# Create Lables
label_encoder = LabelEncoder()

train_y = label_encoder.fit_transform((df_train['rate'] >= 0).astype(int))
eval_y = label_encoder.fit_transform((df_eval['rate'] >= 0).astype(int))
test_y = label_encoder.fit_transform((df_test['rate'] >= 0).astype(int))

### Model General Config

In [ ]:
# Model Config
MAX_LEN = 128
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
TEST_BATCH_SIZE = 16

EPOCHS = 12
# Every EEVERY_EPOCH print status
EEVERY_EPOCH = 1000
LEARNING_RATE = 2e-5
CLIP = 0.0

MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'
OUTPUT_PATH = '/content/bert-fa-base/pytorch_model.bin'

os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

### BERT Tokenizer

In [ ]:
# setup the tokenizer and configuration

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH, return_dict=False)
config = BertConfig.from_pretrained(MODEL_NAME_OR_PATH)

### Sample

In [ ]:
sample = 'از این محصول بدم اومده!'

In [ ]:
tokens = tokenizer.tokenize(sample)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f'Tokens: {tokenizer.convert_tokens_to_string(tokens)}')
print(f'Token IDs: {token_ids}')

In [ ]:
encoding = tokenizer.encode_plus(
    sample,
    max_length=32,
    truncation=True,
    add_special_tokens=True,
    return_token_type_ids=True,
    return_attention_mask=True,
    padding='max_length',
    return_tensors='pt'
)

print(f'Keys: {encoding.keys()}\n')
for k in encoding.keys():
    print(f'{k}:\n{encoding[k]}')

### Load Torch Dataset

In [ ]:
class SentimentDataset(torch.utils.data.Dataset):
    """ Create a PyTorch dataset for Digikala SentimentDataset. """

    def __init__(self, tokenizer, comments, targets, is_predict=False, max_len=128):
        self.comments = comments
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.is_predict = is_predict

    def __len__(self):
        return len(self.comments)

    def __getitem__(self, item):
        comment = str(self.comments[item])

        encoding = self.tokenizer.encode_plus(
            comment,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt')
        
        inputs = {}
        
        inputs = {
            'comment': comment,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }

        if not self.is_predict:
            inputs['targets'] = torch.tensor(self.targets[item], dtype=torch.long)

        return inputs

In [ ]:
def create_data_loader(x, y, tokenizer, max_len, batch_size, is_predict=False):
    dataset = SentimentDataset(
        comments=x,
        targets=y,
        tokenizer=tokenizer,
        max_len=max_len,
        is_predict=is_predict)
    
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [ ]:
train_data_loader = create_data_loader(df_train['comment'].to_numpy(), train_y, tokenizer, MAX_LEN, TRAIN_BATCH_SIZE)
valid_data_loader = create_data_loader(df_eval['comment'].to_numpy(), eval_y, tokenizer, MAX_LEN, VALID_BATCH_SIZE)

### Sample Batch

In [ ]:
# We will make prediction over sample batch for example
sample_batch = next(iter(train_data_loader))
sample_batch

### Sentiment Model

In [ ]:
class SentimentModel(nn.Module):

    def __init__(self, config):
        super(SentimentModel, self).__init__()

        self.bert = BertModel.from_pretrained(MODEL_NAME_OR_PATH, return_dict=False)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        _, pooled_output = self.bert(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            token_type_ids=token_type_ids)
        
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits 

In [ ]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()
pt_model = None

!nvidia-smi

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
pt_model = SentimentModel(config=config)
pt_model = pt_model.to(device)

### Make Prediction Based on Sample Batch
Here is only BERT Model result over sample batch without any fine-tuning

In [ ]:
result = pt_model(sample_batch['input_ids'].to(device), 
                   sample_batch['attention_mask'].to(device),
                   sample_batch['token_type_ids'].to(device))

# Make Prediction
_, predictions = torch.max(result, dim=1)

for i, comment in enumerate(sample_batch['comment']):
    print(str(comment) + " : " + str(predictions[i]))

In [ ]:
def stack_y(y_true, y_pred):
    y_true = torch.stack(y_true).cpu().detach().numpy()
    y_pred = torch.stack(y_pred).cpu().detach().numpy()
    return [y_true, y_pred]

### Train Method

In [ ]:
import collections
from tqdm.notebook import tqdm

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [ ]:
def train(model, 
          data_loader, 
          step=0, 
          print_every_step=100, 
          eval_min_loss=np.Inf,
          eval_data_loader=None, 
          clip=0.0):
    

    # Tell Model that We are training model
    model.train()

    # Store Losses in each step
    losses = []
    # Each Step Prediction
    y_pred = []
    # Each Step True Label
    y_true = []

    # AdamW Optimizer
    optimizer = AdamW(pt_model.parameters(), lr=LEARNING_RATE, correct_bias=False)

    # Loss Function
    loss_fn = nn.CrossEntropyLoss()

    # Total Required Steps
    total_steps = len(train_data_loader) * EPOCHS

    # Scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0,
                                                num_training_steps=total_steps)
    
    for batch in tqdm(data_loader, total=len(data_loader), desc="Training Process "):
        step += 1
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        token_type_ids = batch['token_type_ids']
        targets = batch['targets']

        # Using CUDA if available
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        targets = targets.to(device)

        # Clear previous gradiant optimized variables
        optimizer.zero_grad()

        # Predict results
        results = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)
        
        # convert output probabilities to predicted class
        _, predictions = torch.max(results, dim=1)

        # calculate the batch loss and append it to end of Losses list
        loss = loss_fn(results, targets)
        losses.append(loss.item())
        
        # Also Append predictions and true labels
        y_pred.extend(predictions)
        y_true.extend(targets)

        # Compute gradient of the loss with respect to model parameters
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        if clip > 0.0:
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip)

        # perform optimization step
        optimizer.step()

        # perform scheduler step
        scheduler.step()

        # Stack True labels and Predictions
        train_y = stack_y(y_true, y_pred)

        # Calculate Average Loss
        train_loss = np.mean(losses)
        
        if step % print_every_step == 0:
            eval_y, eval_loss = evaluation(model, eval_data_loader, loss_fn)
            eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')

            print('Epoch: {}/{}...'.format(epoch, epochs))
            print('Step: {}...'.format(step))
            
            print('Train Loss: {:.6f}...'.format(train_loss))
            print('Train Acc: {:.3f}...'.format(accuracy_score(train_y[0], train_y[1])))

            print('Valid Loss: {:.6f}...'.format(eval_loss))
            print('Valid Acc: {:.3f}...'.format(accuracy_score(eval_y[0], eval_y[1])))


            if eval_loss <= eval_min_loss:
                print('Validation Loss Change is : ({:.5f} --> {:.5f}).'.format(eval_loss_min, eval_loss))
                eval_min_loss = eval_loss

                print("Model Saved.")
                torch.save(model.state_dict(), output_path)

    # Stack True labels and Predictions
    train_y = stack_y(y_true, y_pred)

    # Calculate Average Loss
    train_loss = np.mean(losses)

    return train_y, train_loss, step, eval_min_loss

### Evaluation Method

In [ ]:
def evaluation(model, data_loader, loss_fn):

    # Tell Model that We are evaluating model
    model.eval()

    # Store Losses in each step
    losses = []
    # Each Step Prediction
    y_pred = []
    # Each Step True Label
    y_true = []


    # Stop Optimizing on evaluation data
    with torch.no_grad():
        for batch in tqdm(data_loader, total=len(data_loader), desc="Evaluation Process"):
            
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            token_type_ids = batch['token_type_ids']
            targets = batch['targets']

            # Using CUDA if available
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            targets = targets.to(device)

            # Predict results
            results = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
            
            # convert output probabilities to predicted class
            _, predictions = torch.max(results, dim=1)

            # calculate the batch loss and append it to end of Losses list
            loss = loss_fn(results, targets)
            losses.append(loss.item())
            
            # Also Append predictions and true labels
            y_pred.extend(predictions)
            y_true.extend(targets)
    
    # Stack True labels and Predictions
    eval_y = stack_y(y_true, y_pred)

    # Calculate Average Loss
    eval_loss = np.mean(losses)

    return eval_y, eval_loss

### Training Model

In [ ]:
step = 0
eval_min_loss = np.Inf
history = collections.defaultdict(list)

for epoch in tqdm(range(1, EPOCHS + 1), desc="epochs..."):
    train_y, train_loss, step, eval_min_loss = train(
        model=pt_model, 
        data_loader=train_data_loader, 
        step=step, 
        print_every_step=EEVERY_EPOCH, 
        eval_min_loss=eval_min_loss,
        eval_data_loader=valid_data_loader,
        clip=CLIP)
    
    history['train_accuracy'].append(accuracy_score(train_y[0], train_y[1]))
    history['train_f1'].append(f1_score(train_y[0], train_y[1], average='weighted'))
    history['train_loss'].append(train_loss)

    eval_y, eval_loss = evaluation(
        model=pt_model, 
        data_loader=valid_data_loader, 
        loss_fn=nn.CrossEntropyLoss())
    
    history['eval_accuracy'].append(accuracy_score(eval_y[0], eval_y[1]))
    history['eval_f1'].append(f1_score(eval_y[0], eval_y[1], average='weighted'))
    history['eval_loss'].append(eval_loss)

### Predict Method

In [ ]:
def predict(model, comments, tokenizer, max_len=256, batch_size=32):
    data_loader = create_data_loader(comments, None, tokenizer, max_len, batch_size, is_predict=True)
    
    # Append predictions into this list
    predictions_list = []
    
    model.eval()
    with torch.no_grad():
        for batch in tqdm(data_loader, position=0):
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            token_type_ids = batch['token_type_ids']

            # Using CUDA if available
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            
            # Predict results
            results = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
            
            # convert output probabilities to predicted class
            _, predictions = torch.max(results, dim=1)

            predictions_list.extend(predictions)

    predictions = torch.stack(predictions_list).cpu().detach().numpy()

    return predictions

### Predict Test Data

In [ ]:
test_comments = df_test['comment'].to_numpy()
predictions = predict(pt_model, test_comments, tokenizer, max_len=128)

### Accuracy

In [ ]:
print('Accuracy: {}'.format(accuracy_score(test_y, predictions)))

### F1-Score

In [ ]:
print('F1: {}'.format(f1_score(test_y, predictions, average="weighted")))

### Classification Report

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(test_y, predictions, target_names=['positive','negative']))